# Machine Learning Classification

*Author: Evan Carey + Dhamodhar Reddy Atla*

*Copyright 2017-2019, BH Analytics, LLC*

## Overview

The purpose of this section is to go over machine learning! We will focus on classification in the context of python (the scikit-learn module). We will include some general concepts of machine learning as well as the specifics of a few different classification algorithms. For further reading, I highly recommend the free ebook titled 'Introduction to Statistical Learning' by Gareth James. A quick web search should find this book near the top of the search results. For even more in-depth coverage of machine learning algorithms, I recommend the book  'Elements of Statistical Learning' by Trevor Hastie (also free online). 

## Classification

In the case where our outcome (target) variable is discrete with a limited number of possible values, we can use classification algorithms to predict the outcome. Imagine a binary outcome with values of 'Yes' and 'No'. We are interested in predicting the probability that the outcome is either 'Yes' or 'No'. It is also possible to predict outcomes with more than two possible values, but we will focus on the binary case here. 

## Libraries

In [7]:
## Import Modules
import os
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.metrics import confusion_matrix
import sklearn
from sklearn import datasets
from sklearn.model_selection import train_test_split
import time

In [8]:
## Get Version information
print(sys.version)
print("Pandas version: {0}".format(pd.__version__))
print("Matplotlib version: {0}".format(matplotlib.__version__))
print("Numpy version: {0}".format(np.__version__))
print("SciKitLearn version: {0}".format(sklearn.__version__))

3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]
Pandas version: 2.2.2
Matplotlib version: 3.9.2
Numpy version: 1.26.4
SciKitLearn version: 1.5.1


## Patient Mortality Dataset

We will use a dataset with a binary outcome of mortality as a motivating example.

This is a dataset of patients demographics and disease status, with mortality indicated. The dataset is here: 

`data\healthcare\patientAnalyticFile.csv`

In practice, you most likely would have created a dataset like this from multiple other files after cleaning, reshaping, and joining them. 

You can generalize this setup to any situation with a binary outcome, such as estimating the probability of a customer filing a warranty claim, or the probability of a transaction being fraudulent. 

We will first import this dataset and examine the potential variables to use in our classification algorithm.

In [11]:
## Set print limits
pd.options.display.max_rows = 10
## Import Data
df_patient = pd.read_csv('./PatientAnalyticFile.csv')
df_patient

,PatientID,DateOfBirth,Gender,Race,Myocardial_infarction,Congestive_heart_failure,Peripheral_vascular_disease,Stroke,Dementia,Pulmonary,...,Metastatic_solid_tumour,HIV,Obesity,Depression,Hypertension,Drugs,Alcohol,First_Appointment_Date,Last_Appointment_Date,DateOfDeath
0,1,1962-02-27,female,hispanic,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2013-04-27,2018-06-01,NaN
1,2,1959-08-18,male,white,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2005-11-30,2008-11-02,2008-11-02
2,3,1946-02-15,female,white,0,0,0,0,0,0,...,0,1,0,0,1,0,0,2011-11-05,2015-11-13,NaN
3,4,1979-07-27,female,white,0,0,0,0,0,1,...,0,0,0,0,0,0,0,2010-03-01,2016-01-17,2016-01-17
4,5,1983-02-19,female,hispanic,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2006-09-22,2018-06-01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,1997-12-19,female,other,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2008-06-14,2018-06-01,NaN
19996,19997,1984-03-31,female,white,0,0,0,0,0,0,...,0,1,0,0,1,0,0,2007-04-24,2018-06-01,NaN
19997,19998,1993-07-04,female,white,0,0,0,0,0,0,...,0,0,1,0,1,0,0,2010-10-16,2018-06-01,NaN
19998,19999,1984-04-17,male,other,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2015-01-04,2018-06-01,NaN


We need to make a variable to indicate mortality. We can do that based on the abscence of 'date of death':

In [13]:
# Create mortality variable
df_patient['mortality'] = \
    np.where(df_patient['DateOfDeath'].isnull(),
             0,1)

# Convert dateofBirth to date
df_patient['DateOfBirth'] =  pd.to_datetime(df_patient['DateOfBirth'])


# Calculate age in years as of 2015-01-01
df_patient['Age_years'] = ((pd.to_datetime('2015-01-01') - df_patient['DateOfBirth']).dt.days/365.25)

In [14]:
df_patient.describe()

,PatientID,DateOfBirth,Myocardial_infarction,Congestive_heart_failure,Peripheral_vascular_disease,Stroke,Dementia,Pulmonary,Rheumatic,Peptic_ulcer_disease,...,LiverSevere,Metastatic_solid_tumour,HIV,Obesity,Depression,Hypertension,Drugs,Alcohol,mortality,Age_years
count,20000.000000,20000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,...,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,10000.500000,1967-10-02 20:38:36.960000,0.045600,0.043450,0.023950,0.028650,0.031400,0.072650,0.012300,0.009650,...,0.051450,0.033150,0.006450,0.163450,0.106300,0.302900,0.040050,0.079750,0.354700,47.247474
min,1.000000,1936-04-04 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15.753593
25%,5000.750000,1952-01-29 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,31.733744
50%,10000.500000,1967-11-26 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,47.099247
75%,15000.250000,1983-04-08 06:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,62.924025
max,20000.000000,1999-04-01 00:00:00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,78.743326
std,5773.647028,NaN,0.208621,0.203873,0.152897,0.166825,0.174401,0.259568,0.110224,0.097762,...,0.220919,0.179033,0.080054,0.369785,0.308229,0.459524,0.196081,0.270913,0.478434,18.145086


We should change date of birth to be an actual date and calculate age if we want to include it in the model:

## Workflow into scikit-learn


* There are a number of possible ways to prepare data for modeling in scikit-learn. 
* You must end up with a numeric ndarray of inputs (X) and a numeric ndarray matrix of the target (Y)
* I prefer the following workflow:
  * We use pandas to import and clean data
  * We use Patsy to create the X and Y ndarrays
  * Using categorical transformations (dummy coding) as needed
  * Also can generate non-linear terms including splines
  * Use scikit-learn for machine learning

## Use Patsy to Create the Model Matrices

We typically start out with a pandas dataframe for manipulation purposes, then we will use this dataframe as the input to the machine learning library. I created a pandas dataframe above to replicate this process. We will use the dmatrices function from the patsy library to easily generate the design matrices for the machine learning algorithms representing the inputs. THis handles the following:

* drops rows with missing data
* construct one-hot encoding for categorical variables
* optionally adds constant intecercept

In [18]:
## Create formula for all variables in model
vars_remove = ['PatientID','First_Appointment_Date','DateOfBirth',
               'Last_Appointment_Date','DateOfDeath','mortality']
vars_left = set(df_patient.columns) - set(vars_remove)
formula = "mortality ~ " + " + ".join(vars_left)
formula

'mortality ~ Renal + HIV + Age_years + Cancer + Diabetes_with_complications + Hypertension + Gender + Depression + Peripheral_vascular_disease + Race + Pulmonary + Metastatic_solid_tumour + Diabetes_without_complications + Drugs + Rheumatic + LiverSevere + Alcohol + Dementia + Paralysis + Peptic_ulcer_disease + Stroke + Myocardial_infarction + LiverMild + Congestive_heart_failure + Obesity'

In [19]:
## only use subset of data so models fit in reasonable time
df_patient_sub = df_patient.sample(frac=0.1,
                     random_state=32)    


## use Patsy to create model matrices
Y,X = dmatrices(formula,
                df_patient_sub)

## Split into Testing and Training Samples

* The first step is to set aside a test sample of data that will allow us to estimate the generalization error post-fit. This protects against overfitting. 
* We can use “tuple unpacking” to assign the values (very pythonic :)
* We can assign a random seed (state) and fraction to split.

 For simple random splits, scikit-learn has a function `train_test_split()`

In [21]:
## Split Data into training and sample

X_train, X_test, y_train, y_test = train_test_split(X,
                     np.ravel(Y), # prevents dimensionality error later!
                     test_size=0.20,
                     random_state=42)

## Confirm the Output Dimensions

* We can confirm the dimensions of the data are the same within test and train
* The proportion should also be close to the test_size argument. 

## First Model: Logistic Regression

* We will start with a basic logistic regression model.
* The flow will be similar for other models
* Call and save model object with initial parameters, then call the fit() method to perform the optimization
* Then call other summary methods post fit to explore the model

Check the docs: 

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [25]:
## import linear model
from sklearn import linear_model
## Define model parameters
## can implement penalties, but check docs for appropriate solver
clf = linear_model.LogisticRegression(fit_intercept=True, # already have the intercept
                                      solver='liblinear') # could change to lbfgs!
## fit model using data with .fit
clf.fit(X_train,y_train)

LogisticRegression(solver='liblinear')

How do we know if this is a good model? What makes a good model? Let's make predictions, is this a good model? Which parameters are most important?

In [27]:
## get accuracy
sklearn.metrics.accuracy_score(y_train,
                               clf.predict(X_train))

0.73125

In [28]:
## Create dict to store all these results:
result_scores = {}
## Score the Model on Training and Testing Set
result_scores['Logistic'] = (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))

In [29]:
## Create Function to Print Results
def get_results(x1):
    print("\n{0:20}   {1:4}    {2:4}".format('Model','Train','Test'))
    print('-------------------------------------------')
    for i in x1.keys():
        print("{0:20}   {1:<6.4}   {2:<6.4}".format(i,x1[i][0],x1[i][1]))

In [30]:
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7312   0.7375


## Comparison to the Null Model

Is that a good score for accuracy? Compared to what? We can consider a null model of simply predicting the most frequent class as a base model. Without any other information, I may predict based simply on the distribution of the outcome.

Scikitlearn has a built in dummy classifier that works similarly:

In [34]:
## Dummy classifier
from sklearn.dummy import DummyClassifier
clf = DummyClassifier(strategy='most_frequent',
                      random_state=0)
clf.fit(X_train, y_train)
clf.score(X_train, y_train)  

0.64375

In [35]:
## Score the Model on Training and Testing Set
result_scores['Null'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))

In [36]:
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7312   0.7375
Null                   0.6438   0.61  


In [37]:
#### Fit Random Forest
## Random Forests
from sklearn import ensemble
clf = ensemble.RandomForestClassifier(n_estimators=100, 
                                      max_features=10,
                                      random_state=42)
clf.fit(X_train,y_train)
## get confusion matrix
confusion_matrix(y_train,clf.predict(X_train))

array([[1030,    0],
       [   1,  569]], dtype=int64)

In [38]:
## Score the Model on Training and Testing Set
result_scores['RandomForest_noCV'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7312   0.7375
Null                   0.6438   0.61  
RandomForest_noCV      0.9994   0.6975


## Grid Search for Manual Cross Validation

There is no RandomForestCV function....what to do? 


We can specify a grid search across a range of hyperparameters. 

In [41]:
from sklearn.model_selection import GridSearchCV
## specify grid
parameters = {'n_estimators':(50,100,200,300),
              'max_features':(5,10,15,20)}
## specify model without hyperparameters
rf_model = ensemble.RandomForestClassifier(random_state=32)
## specify search with model
clf = GridSearchCV(rf_model,
                   parameters,
                   cv=5,
                   return_train_score=True)
clf.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=32),
             param_grid={'max_features': (5, 10, 15, 20),
                         'n_estimators': (50, 100, 200, 300)},
             return_train_score=True)

In [42]:
## add model score
## Score the Model on Training and Testing Set
result_scores['RandomForest_CV'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7312   0.7375
Null                   0.6438   0.61  
RandomForest_noCV      0.9994   0.6975
RandomForest_CV        0.9994   0.685 


In [43]:
from sklearn.model_selection import GridSearchCV
## specify grid
parameters2 = {'max_depth':(2,5,7,10,20)}
## specify model without hyperparameters
rf_model = ensemble.RandomForestClassifier(max_features=20,
                                           n_estimators=100,
                                           random_state=32)
## specify search with model
clf = GridSearchCV(rf_model,
                   parameters2,
                   cv=5,
                   return_train_score=True)
clf.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(max_features=20, random_state=32),
             param_grid={'max_depth': (2, 5, 7, 10, 20)},
             return_train_score=True)

In [44]:
## add model score
## Score the Model on Training and Testing Set
result_scores['RandomForest_CV2'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))

## Regularized Linear Regression

- The next family of models we will consider are called regularized linear regression. 
- This includes LASSO, Elastic Net, and Ridge regression. 
- These are penalized forms of a regular linear regression (like a logistic regression). 
- The basic idea is that we can place a penalty on the estimated coefficients from the general linear model, 'pushing' them towards zero. 
- If the coefficients are related to the outcome, they will 'push' back against our penalty. 
- The stronger the relationship (or the stronger the predictor), the stronger they will 'push' back. 
- The overall effect is that the coefficients are all shrunk towards zero. If the variable is not strongly related to the outcome, it will be shrunk close to zero, or possibly all the way to zero. 
- This can give us effective variable selection, where the weak variables are eliminated since their coefficients are shrunk all the way to zero. 
- Depending on how we apply the penalty, variables will either be shrunk all the way to zero (this is called the LASSO), or they will be shrunk to a small number, but still above zero (This is called ridge regression).
- We can also apply a mixture of the two penalties, which is called the elastic net regression. 
- A natural question you might ask is, how do I pick the best model?
    + LASSO?
    + Ridge regression?
    + Elastic net (the mixture of the two)?
- Also, how strong of a penalty should I pick?
    + A very weak penalty, so it is essentially just a logistic regression?
    + A very strong penalty, so almost all the coefficients are equal to zero?
    + Maybe something in between?

In [46]:
solvers = ['liblinear', 'lbfgs', 'newton-cg', 'sag', 'saga']
results = []

for solver in solvers:
    try:
        # Initialize the model
        clf = linear_model.LogisticRegression(solver=solver, max_iter=5000)

        # Record start time
        start_time = time.time()

        # Fit the model
        clf.fit(X_train, y_train)

        # Record end time
        end_time = time.time()

        # Calculate accuracies
        train_accuracy = sklearn.metrics.accuracy_score(y_train, clf.predict(X_train))
        test_accuracy = sklearn.metrics.accuracy_score(y_test, clf.predict(X_test))
        time_taken = end_time - start_time

        # Store results
        results.append([solver, train_accuracy, test_accuracy, time_taken])

        result_scores[f"Logistic_{solver}"] = (train_accuracy, test_accuracy)
    except Exception as e:
        # Handle solver-related errors
        results.append([solver, "Error", "Error", str(e)])

C:\Users\dhamo\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\dhamo\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


## Question 2: Next, fit a series of logistic regression models, without regularization. Each model should use the same set of predictors (all of the relevant predictors in the dataset) and should use the entire dataset, rather than a fraction of it. Use a randomly chosen 80% proportion of observations for training and the remaining for checking the generalizable performance (i.e., performance on the holdout subset). Be sure to ensure that the training and holdout subsets are identical across all models. Each model should choose a different solver.

In [48]:
# Create a results DataFrame
results_df = pd.DataFrame(results, columns=['Solver used', 'Training subset accuracy',
                                            'Holdout subset accuracy', 'Time taken (seconds)'])

## Question 3: Compare the results of the models in terms of their accuracy (use this as the performance metric to assess generalizability error on the holdout subset) and the time taken (use appropriate timing function). Summarize your results via a table with the following structure:

In [49]:
results_df

,Solver used,Training subset accuracy,Holdout subset accuracy,Time taken (seconds)
0,liblinear,0.731250,0.7375,0.016515
1,lbfgs,0.730625,0.7375,0.368095
2,newton-cg,0.730625,0.7375,0.043258
3,sag,0.730625,0.7375,3.745809
4,saga,0.730625,0.7375,4.399826


# Answering Questions as per Assignment

## Question 1: Among the different classification models included in the Python notebook, which model had the best overall performance? Support your response by referencing appropriate evidence.

In [47]:
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7312   0.7375
Null                   0.6438   0.61  
RandomForest_noCV      0.9994   0.6975
RandomForest_CV        0.9994   0.685 
RandomForest_CV2       0.7556   0.7225
Logistic_liblinear     0.7312   0.7375
Logistic_lbfgs         0.7306   0.7375
Logistic_newton-cg     0.7306   0.7375
Logistic_sag           0.7306   0.7375
Logistic_saga          0.7306   0.7375


Among the different classification models, the logistic regression models demonstrated the best overall performance. All variants of logistic regression, including the base model and solvers like liblinear, lbfgs, newton-cg, sag, and saga, achieved the highest test accuracy of 0.7375. This indicates strong generalization to unseen data and consistent performance across solver types. In contrast, while the Random Forest models achieved nearly perfect training accuracies (0.9994), their test accuracies were significantly lower—0.6975 for RandomForest_noCV, 0.685 for RandomForest_CV, and 0.7225 for RandomForest_CV2—suggesting overfitting. The null model performed the worst, with a test accuracy of only 0.61, serving as a baseline for comparison. Therefore, the logistic regression models, with their balance of strong test performance and lack of overfitting, stand out as the best-performing models overall.

The Logistic Regression models had the best overall performance, as they achieved the highest and most consistent test accuracy (0.7375) without signs of overfitting, making them the most reliable choice for generalization.

## Question 4: Based on the results, which solver yielded the best results? Explain the basis for ranking the models - did you use training subset accuracy? Holdout subset accuracy? Time of execution? All three? Some combination of the three?

The **liblinear solver** yielded the best overall performance. While all solvers achieved the same holdout subset accuracy of **0.7375**, which is the most important metric for evaluating a model's generalization ability, the liblinear solver stood out due to its **higher training accuracy (0.731250)** and **significantly lower execution time (0.0165 seconds)** compared to the others. Although the differences in training accuracy among the solvers were minimal, liblinear had a slight edge, and its speed advantage was substantial, especially when compared to solvers like saga and sag, which took over 3 seconds to run. In ranking the models, the primary criterion was **holdout subset accuracy**, since it reflects performance on unseen data. However, **training accuracy and execution time** were also considered to break ties. Given that all solvers performed equally well on the holdout set, the combination of **higher training accuracy and much faster computation time** made liblinear the most efficient and effective solver among the options.